In [2]:
import pandas as pd
import numpy as np
import os

# import time
import multiprocessing as mp
import glob
import sys 
import sys  
sys.path.insert(0, '../')

from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from muwclass_library import loo_prepare, loo_train_and_classify, loo_save_res, plot_confusion_matrix,  confident_flag, confident_sigma
from bokeh.io import show
from bokeh.io import export_png
from sklearn.metrics import confusion_matrix
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
dir_data = '/media/orion51/PoplarII/Projects/2019/MUWCLASS_Project/ML/DATA/MUWCLASS_Classify_evaluation/LOO_test_rgeo_lum_results'
class_labels = ['AGN','CV','HM-STAR','LM-STAR','HMXB','LMXB','NS','YSO']

files = glob.glob(f'{dir_data}/*/classes.csv')
dfs = [pd.read_csv(f, usecols=range(len(class_labels))) for f in files]
print(len(dfs))
df_median   = pd.concat(dfs).groupby(level=0).median()
#df_1sig_up  = pd.concat(dfs).groupby(level=0).quantile(0.84135)
#df_1sig_low = pd.concat(dfs).groupby(level=0).quantile(0.15865)
df_2sig_up  = pd.concat(dfs).groupby(level=0).quantile(0.97725)
df_2sig_low = pd.concat(dfs).groupby(level=0).quantile(0.02275)
df_mean     = pd.concat(dfs).groupby(level=0).mean()
df_std      = pd.concat(dfs).groupby(level=0).std()


df_class = df_median.idxmax(axis=1)
df_prob = df_median.max(axis=1)
#df_prob_1sig_up  = pd.DataFrame(data=[df_1sig_up.values[i][np.argmax(np.array(df_median), axis=1)[i]]  for i in range(len(df_median))], columns=['Class_prob_1sig_upp'])
#df_prob_1sig_low = pd.DataFrame(data=[df_1sig_low.values[i][np.argmax(np.array(df_median), axis=1)[i]]  for i in range(len(df_median))], columns=['Class_prob_1sig_low'])
df_prob_2sig_up  = pd.DataFrame(data=[df_2sig_up.values[i][np.argmax(np.array(df_median), axis=1)[i]]  for i in range(len(df_median))], columns=['Class_prob_2sig_upp'])
df_prob_2sig_low = pd.DataFrame(data=[df_2sig_low.values[i][np.argmax(np.array(df_median), axis=1)[i]]  for i in range(len(df_median))], columns=['Class_prob_2sig_low'])
df_class_mean = df_mean.idxmax(axis=1)
df_prob_mean = df_mean.max(axis=1)
df_prob_e = pd.DataFrame(data=[df_std.values[i][np.argmax(np.array(df_mean), axis=1)[i]]  for i in range(len(df_std))], columns=['Class_prob_e'])

df_mean = df_mean.add_prefix('P_')
df_std  = df_std.add_prefix('e_P_')
df_median = df_median.add_prefix('P_median_')
df_2sig_up = df_2sig_up.add_prefix('P_2sig_upp_')
df_2sig_low = df_2sig_low.add_prefix('P_2sig_low_')

df = pd.concat([df_median, df_2sig_up, df_2sig_low, df_mean, df_std, df_class, df_prob, df_prob_2sig_up, df_prob_2sig_low, df_class_mean, df_prob_mean, df_prob_e, pd.read_csv(files[0]).name, pd.read_csv(files[0]).true_Class],axis=1).rename(columns={0:'Class_median',1:'Class_prob_median',2:'Class',3:'Class_prob'})
#print(df.columns)

CO_classes = ['LMXB', 'HMXB', 'CV', 'NS']
nonCO_classes = list(set(class_labels)-set(CO_classes))

# define probability and error of stellar compact object 

df['P_CO'] = df[['P_' + c for c in CO_classes]].sum(1)
df['e_P_CO'] = np.sqrt(np.sum(np.square(df[['e_P_' + c for c in CO_classes]]), 1))



#df_default = confident_flag(df, method = 'sigma-mean', class_cols=class_labels)
df = confident_sigma(df, class_cols=class_labels).rename(columns={'CT':'CT_8'})

df_mean_5class = df[['P_AGN','P_LM-STAR','P_HM-STAR','P_YSO','P_CO']]
df_std_5class = df[['e_P_AGN','e_P_LM-STAR','e_P_HM-STAR','e_P_YSO','e_P_CO']]
df_class_mean = df_mean_5class.idxmax(axis=1)
df_prob_mean = df_mean_5class.max(axis=1)
df_prob_e = pd.DataFrame(data=[df_std_5class.values[i][np.argmax(np.array(df_mean_5class), axis=1)[i]]  for i in range(len(df_std_5class))], columns=['Class5_prob_e'])

#df_class_mean.columns = df_class_mean.columns.map(lambda x: x.removeprefix("P_"))

df = pd.concat([df,df_class_mean, df_prob_mean, df_prob_e],axis=1).rename(columns={0:'Class5',1:'Class5_prob'})
#print(df)
df['Class5'] = df['Class5'].str[2:]
df['true_Class5'] = df.apply(lambda s:s['true_Class'] if s['true_Class'] in ['AGN','LM-STAR','HM-STAR','YSO'] else 'CO', axis=1)
print(df)
df = confident_sigma(df, class_cols=['AGN','LM-STAR','HM-STAR','YSO','CO'],class_prob='Class5_prob',class_prob_e='Class5_prob_e').rename(columns={'CT':'CT_5'})

df.to_csv(f'{dir_data}/LOO_classes.csv',index=False)



1000
      P_median_AGN  P_median_CV  P_median_HM-STAR  P_median_HMXB  \
0            0.980         0.00              0.00           0.00   
1            1.000         0.00              0.00           0.00   
2            0.980         0.01              0.00           0.00   
3            0.980         0.01              0.00           0.00   
4            0.990         0.00              0.00           0.00   
...            ...          ...               ...            ...   
2936         0.030         0.00              0.01           0.01   
2937         0.320         0.00              0.00           0.00   
2938         0.000         0.00              0.00           0.00   
2939         0.000         0.00              0.00           0.00   
2940         0.075         0.23              0.01           0.04   

      P_median_LM-STAR  P_median_LMXB  P_median_NS  P_median_YSO  \
0                 0.01           0.00         0.00          0.00   
1                 0.00           0.00     

In [8]:
df = pd.read_csv(f'./data/LOO_classes.csv')
title = 'CM'
class_labels = ['AGN','HM-STAR','LM-STAR','YSO','CV','HMXB','LMXB','NS']

print('accuracy for all classifications:',accuracy_score(df.true_Class, df.Class))
print('accuracy for confident classifications:',accuracy_score(df.loc[df.CT_8>=2.0, 'true_Class'], df.loc[df.CT_8>=2.0, 'Class']))
print('balanced accuracy for all classifications:',balanced_accuracy_score(df.true_Class, df.Class))
print('balanced accuracy for confident classifications:',balanced_accuracy_score(df.loc[df.CT_8>=2.0, 'true_Class'], df.loc[df.CT_8>=2.0, 'Class']))


CM = plot_confusion_matrix(df, title=f'Recall {title}',classes=class_labels)
#show(CM)
conf_CM = plot_confusion_matrix(df[df.CT_8>=2.0], title=f'Confident Recall {title}',classes=class_labels,count_fraction=True,df_all = df)
CM_p = plot_confusion_matrix(df, title=f'Precision {title}',classes=class_labels,cm_type='precision')
#show(CM)
conf_CM_p = plot_confusion_matrix(df[df.CT_8>=2.0], title=f'Confident Precision {title}',classes=class_labels,cm_type='precision',count_fraction=True,df_all = df)
export_png(CM, filename=f'./figs/CM_recall.png')
export_png(conf_CM, filename=f'./figs/CM_recall_conf.png')
export_png(CM_p, filename=f'./figs/CM_precision.png')
export_png(conf_CM_p, filename=f'./figs/CM_precision_conf.png')

accuracy for all classifications: 0.8956137368242094
accuracy for confident classifications: 0.9672
balanced accuracy for all classifications: 0.7285953153070455
balanced accuracy for confident classifications: 0.8142204292284292


'/home/orion51/Desktop/Research/MUWCLASS/MUWCLASS_CSCv2/codes/NGC3532/figs/CM_precision_conf.png'

In [7]:

title = 'CM'
class5_labels = ['AGN','HM-STAR','LM-STAR','YSO','CO']


print('accuracy for all classifications:',accuracy_score(df.true_Class5, df.Class5))
print('accuracy for confident classifications:',accuracy_score(df.loc[df.CT_5>=2.0, 'true_Class5'], df.loc[df.CT_5>=2.0, 'Class5']))
print('balanced accuracy for all classifications:',balanced_accuracy_score(df.true_Class5, df.Class5))
print('balanced accuracy for confident classifications:',balanced_accuracy_score(df.loc[df.CT_5>=2.0, 'true_Class5'], df.loc[df.CT_5>=2.0, 'Class5']))


CM = plot_confusion_matrix(df, title=f'5-class Recall {title}',classes=class5_labels,true_class='true_Class5',pred_class='Class5')
#show(CM)
conf_CM = plot_confusion_matrix(df[df.CT_5>=2.0], title=f'5-class Confident Recall {title}',classes=class5_labels,true_class='true_Class5',pred_class='Class5',count_fraction=True,df_all = df)
CM_p = plot_confusion_matrix(df, title=f'5-class Precision {title}',classes=class5_labels,true_class='true_Class5',pred_class='Class5',cm_type='precision')
#show(CM)
conf_CM_p = plot_confusion_matrix(df[df.CT_5>=2.0], title=f'5-class Confident Precision {title}',classes=class5_labels,true_class='true_Class5',pred_class='Class5',cm_type='precision',count_fraction=True,df_all = df)
export_png(CM, filename=f'./figs/CM_recall_5class.png')
export_png(conf_CM, filename=f'./figs/CM_recall_conf_5class.png')
export_png(CM_p, filename=f'./figs/CM_precision_5class.png')
export_png(conf_CM_p, filename=f'./figs/CM_precision_conf_5class.png')


accuracy for all classifications: 0.9139748384903095
accuracy for confident classifications: 0.9642718446601942
balanced accuracy for all classifications: 0.8875975904199797
balanced accuracy for confident classifications: 0.9484024337003433


'/home/orion51/Desktop/Research/MUWCLASS/MUWCLASS_CSCv2/codes/NGC3532/figs/CM_precision_conf_5class.png'

# confusion matrix
recall_cm = plot_confusion_matrix(pd.read_csv('../../files/LOO_classes.csv'),class_labels,title='Norm. Confusion Matrix (%)',width=450,height=400)
show(recall_cm)

# feature importance
feaimps = pd.read_csv('../../files/feature_importance.csv')
fea_imp = plot_Feature_Importance_withSTD(feaimps.iloc[0], feaimps.iloc[1],feaimps.columns,400,800)
